---
title: "Module 05: Lab 02"
subtitle: "Regression Modeling on Employment Data"
author:
  - name: Qimin Shen
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: "2025-4-11"
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
date-modified: today
date-format: long
execute: 
  echo: false
  eval: false
  freeze: auto
---

# Objectives {.unnumbered}

1. Use **PySpark** to process the Lightcast dataset.
2. Engineer features from structured columns for salary prediction.
3. Train **Linear Regression model**.
4. Evaluate models using **RMSE** and **R²**.
5. Visualize predictions using diagnostic plots.
6. Push work to GitHub and submit the repository link.

# Setup {.unnumbered}

The instruction below provides you with general keywords for columns used in the lightcast file. See the data schema generated after the load dataset code above to use proper column name. For visualizations, tables, or summaries, please **customize colors, fonts, and styles** as appropriate to avoid a **2.5-point deduction**. Also, **provide a two-sentence explanation** describing key insights drawn from each section's code and outputs. 

1. Follow the steps below as necessary, use your best judgement in importing/installing/creating/saving files as needed.
2. Create a new Jupyter Notebook in your `ad688-sp25-lab08` directory named `lab08_yourname.ipynb`, if the file exists make sure to change the name.
3. Use your **EC2 instance** for this lab.
4. Ensure the `lightcast_data.csv` file is available on the EC2 instance. if not then **Download the dataset**
5. **Add the dataset to `.gitignore`** to avoid pushing large files to GitHub. Open your `.gitignore` file and add:
6. Make sure to create a virtual environment and install the required Python libraries if needed, don't forget to activate it:
7. Install the required Python libraries if needed, you can also use the given requirement file to install the packages to the virtual environment:

```bash
python3 -m venv .venv
source .venv/bin/activate
gdown https://drive.google.com/uc?id=1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ
echo "lightcast_job_postings.csv" >> .gitignore
pip install -r requirements.txt
```


# Load the Dataset
1. **Load the Raw Dataset**:
   - Use Pyspark to the `lightcast_data.csv` file into a DataFrame:
   - You can reuse the previous code. 
   - [Copying code from your friend constitutes plagiarism. DO NOT DO THIS]{.uured-bold}.

In [5]:
#| eval: false
#| echo: true
import os

os.environ["SPARK_HOME"] = "/opt/spark-3.5.5-bin-hadoop3"
os.environ["PATH"] = f"/opt/spark-3.5.5-bin-hadoop3/bin:{os.environ['PATH']}"

from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col, when, regexp_extract
import shutil
import plotly.figure_factory as ff 

spark = SparkSession.builder \
    .appName("LightcastData") \
    .getOrCreate()

data_path = "/home/xn/lightcast_job_postings.csv"
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv(data_path)

df.printSchema() # comment this line when rendering the submission
df.show(5)

root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: date (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: date (nullable = true)
 |-- EXPIRED: date (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: date (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (nullable

# Feature Engineering

Feature Engineering is a crucial step in preparing your data for machine learning. In this lab, we will focus on the following tasks:

1. **Drop rows with missing values** in the target variable and key features.
2. By now you are already familiar with the code and the data. Based on your understanding please choose any 3 (my code output has 10) variables as:
   1. two continuous variables (use your best judgment!)
   2. one categorical.
   3. Your dependent variable (y) is `SALARY`.
3. **Convert categorical variables** into numerical representations using **StringIndexer** and **OneHotEncoder**.
4. **Assemble features** into a single vector using **VectorAssembler**.
5. **Split the data** into training and testing sets.

In [6]:
#| eval: false
#| echo: true
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

selected_cols = ["SALARY", "MIN_YEARS_EXPERIENCE", "SALARY_FROM", "EMPLOYMENT_TYPE_NAME"]
df_clean = df.select(selected_cols).dropna()

indexer = StringIndexer(inputCol="EMPLOYMENT_TYPE_NAME", outputCol="EMPLOYMENT_TYPE_INDEX")
df_indexed = indexer.fit(df_clean).transform(df_clean)

encoder = OneHotEncoder(inputCol="EMPLOYMENT_TYPE_INDEX", outputCol="EMPLOYMENT_TYPE_VEC")
df_encoded = encoder.fit(df_indexed).transform(df_indexed)

assembler = VectorAssembler(
    inputCols=["MIN_YEARS_EXPERIENCE", "SALARY_FROM", "EMPLOYMENT_TYPE_VEC"],
    outputCol="features"
)
df_final = assembler.transform(df_encoded)

final_data = df_final.select(col("features"), col("SALARY").alias("label"))

train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=688)

train_data.show(5)
test_data.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|(4,[1],[30000.0])|68970|
|(4,[1],[35360.0])|35360|
|(4,[1],[37440.0])|37440|
|(4,[1],[37440.0])|37440|
|(4,[1],[65345.0])|80579|
+-----------------+-----+
only showing top 5 rows



+--------------------+-----+
|            features|label|
+--------------------+-----+
|   (4,[1],[41600.0])|73485|
|   (4,[1],[65000.0])|70000|
|[0.0,15000.0,1.0,...|70299|
|[0.0,20800.0,0.0,...|23585|
|[0.0,33176.0,0.0,...|33176|
+--------------------+-----+
only showing top 5 rows



# Train/Test Split

- Perform a **random split** of the data into training and testing sets.
- Set a random seed for reproducibility.
- You can choose a number for splitting to your liking, justify your choice.

In [28]:
#| eval: true
#| echo: false
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=688)
print((train_data.count(), len(train_data.columns)))
print((test_data.count(), len(test_data.columns)))

(18999, 2)


(4705, 2)


The dataset was randomly split into a training set (80%) and a test set (20%) using randomSplit([0.8, 0.2], seed=688). This resulted in 18,999 records for training and 4,705 for testing, totaling 23,704 records.

# Linear Regression

- Train a **Linear Regression** model using the training data. [You will run in to an important issue here. Please make an effort in figuring it by yourself. This is one of the most asked interview questions in CapitalOne's management recruiting program.]{.uured-bold}
- Evaluate the model on the test data.
- Print the coefficients, intercept, R², RMSE, and MAE.
- Use the `summary` object to extract the coefficients and their standard errors, t-values, and p-values.
- Create a DataFrame to display the coefficients, standard errors, t-values, p-values, and confidence intervals.
- Interpret the coefficients and their significance and explain the model performance metrics.

In [10]:
#| eval: false
#| echo: false
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="label")

lr_model = lr.fit(train_data)

test_results = lr_model.evaluate(test_data)

print("R²:", test_results.r2)
print("RMSE:", test_results.rootMeanSquaredError)
print("MAE:", test_results.meanAbsoluteError)

print("Intercept:", lr_model.intercept)
print("Number of coefficients:", len(lr_model.coefficients))

training_summary = lr_model.summary

coefs = lr_model.coefficients
se = training_summary.coefficientStandardErrors
tvals = training_summary.tValues
pvals = training_summary.pValues

import pandas as pd
from pyspark.sql import Row

rows = []
for i in range(len(coefs)):
    rows.append(Row(
        feature_index=i,
        coefficient=float(coefs[i]),
        standard_error=float(se[i]),
        t_value=float(tvals[i]),
        p_value=float(pvals[i]),
        ci_lower=float(coefs[i] - 1.96 * se[i]),
        ci_upper=float(coefs[i] + 1.96 * se[i])
    ))

diagnostic_df = spark.createDataFrame(rows)
diagnostic_df.show(10)

25/04/11 02:12:27 WARN Instrumentation: [feea54db] regParam is zero, which might cause numerical instability and overfitting.
25/04/11 02:12:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/04/11 02:12:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


R²: 0.8412012932179281
RMSE: 17326.42176937338
MAE: 13668.641926030183
Intercept: 22845.780380124743
Number of coefficients: 4


+-------------+------------------+--------------------+-------------------+-------------------+-------------------+------------------+
|feature_index|       coefficient|      standard_error|            t_value|            p_value|           ci_lower|          ci_upper|
+-------------+------------------+--------------------+-------------------+-------------------+-------------------+------------------+
|            0|2683.1576120508757|  40.863560155453946|  65.66137658695315|                0.0|  2603.065034146186|2763.2501899555655|
|            1|0.9548687896340677|0.003665551776995...|  260.4979680348764|                0.0| 0.9476843081511558|0.9620532711169797|
|            2|-9939.179124576016|   941.5318223689148|-10.556392135072846|                0.0|-11784.581496419089|-8093.776752732943|
|            3|-6559.643810275468|  1210.3296726459948| -5.419716593359994|6.04206762400139E-8| -8931.889968661619|-4187.397651889318|
+-------------+------------------+--------------------+

The linear regression model achieved an R² score of 0.841, indicating that approximately 84.1% of the variance in the target variable (salary) is explained by the model.
The RMSE of 17,326 and MAE of 13,668 show that the model has a reasonably low average error when predicting salary values.

The model has 4 coefficients, with each corresponding to a feature in the dataset. Notably: The coefficients for feature 0 and 1 are positive and statistically significant (p < 0.05), indicating a strong positive relationship with salary.The coefficients for features 2 and 3 are negative and significant, suggesting that those categorical levels (from one-hot encoding) are associated with lower expected salary.

All p-values are essentially 0, implying strong statistical significance. Confidence intervals do not cross zero, further confirming the significance of the predictors.

## Generalized Linear Regression Summary
The summary of the Generalized Linear Regression model provides important insights into the model's performance and the significance of each feature. The coefficients indicate the relationship between each feature and the target variable (salary), while the standard errors, t-values, and p-values help assess the reliability of these estimates.

- Please interpret them in the context of your data and model. 
- Feature Names are purposefully not printed in the output. You can use the `features` variable to print them out.

In [11]:
#| eval: true
#| echo: false
assembler = VectorAssembler(
    inputCols=["MIN_YEARS_EXPERIENCE", "SALARY_FROM", "EMPLOYMENT_TYPE_VEC"],
    outputCol="features"
)

features = assembler.getInputCols()

ohe_model = encoder.fit(df_indexed)
category_sizes = ohe_model.categorySizes  

feature_names = ["MIN_YEARS_EXPERIENCE", "SALARY_FROM"] + \
                [f"EMPLOYMENT_TYPE_{i}" for i in range(category_sizes[0])]

coef_info = list(zip(
    feature_names,
    lr_model.coefficients,
    training_summary.coefficientStandardErrors,
    training_summary.tValues,
    training_summary.pValues
))

coef_df = spark.createDataFrame([
    Row(
        feature=feature,
        coefficient=float(coef),
        std_error=float(se),
        t_value=float(tval),
        p_value=float(pval),
        ci_lower=float(coef - 1.96 * se),
        ci_upper=float(coef + 1.96 * se)
    )
    for feature, coef, se, tval, pval in coef_info
])

coef_df.show(10)



+--------------------+------------------+--------------------+-------------------+-------------------+-------------------+------------------+
|             feature|       coefficient|           std_error|            t_value|            p_value|           ci_lower|          ci_upper|
+--------------------+------------------+--------------------+-------------------+-------------------+-------------------+------------------+
|MIN_YEARS_EXPERIENCE|2683.1576120508757|  40.863560155453946|  65.66137658695315|                0.0|  2603.065034146186|2763.2501899555655|
|         SALARY_FROM|0.9548687896340677|0.003665551776995...|  260.4979680348764|                0.0| 0.9476843081511558|0.9620532711169797|
|   EMPLOYMENT_TYPE_0|-9939.179124576016|   941.5318223689148|-10.556392135072846|                0.0|-11784.581496419089|-8093.776752732943|
|   EMPLOYMENT_TYPE_1|-6559.643810275468|  1210.3296726459948| -5.419716593359994|6.04206762400139E-8| -8931.889968661619|-4187.397651889318|
+-----

MIN_YEARS_EXPERIENCE has a positive coefficient (~2683), indicating that each additional year of experience is associated with an increase in salary by about $2,683, holding other factors constant. The low p-value (0.0) confirms that this relationship is statistically significant.

SALARY_FROM also shows a strong positive relationship with the target variable. For every additional dollar in the job’s minimum posted salary, the expected final salary increases by approximately 0.95 dollars. Its extremely small standard error and very high t-value support the reliability of this estimate.

EMPLOYMENT_TYPE_0 and EMPLOYMENT_TYPE_1 (these likely represent specific job types like part-time or contract, from one-hot encoding) have negative coefficients (-9939 and -6559), meaning jobs with these types tend to have lower predicted salaries compared to the baseline type (which was not one-hot encoded).
Their confidence intervals do not cross zero, and their p-values are also highly significant (<< 0.05), showing that these effects are meaningful.

# Diagnostic Plot

Diagnostic plots are essential for evaluating the performance of regression models. In this section, we will create several diagnostic plots to assess the linear regression model's assumptions and performance. There are four (2*2 grid) main plots we will create, you can use `seaborn` or `matplotlib` for this:

1. **Predicted vs Actual Plot**
2. **Residuals vs Predicted Plot**
3. **Histogram of Residuals**
4. **QQ Plot of Residuals**


In [26]:
#| eval: true
#| echo: false
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

predictions = lr_model.transform(test_data)

pred_pd = predictions.select("prediction", "label").toPandas()

pred_pd["residual"] = pred_pd["label"] - pred_pd["prediction"]

sns.set(style="whitegrid")
plt.rcParams.update({'axes.facecolor': 'white'})

pred_pd["residual"] = pred_pd["label"] - pred_pd["prediction"]

fig, axs = plt.subplots(2, 2, figsize=(14, 10))

sns.scatterplot(x="prediction", y="label", data=pred_pd, color="skyblue", alpha=0.6, ax=axs[0, 0])
sns.regplot(x="prediction", y="label", data=pred_pd, scatter=False, ax=axs[0, 0], color="darkred", line_kws={"lw":2})
axs[0, 0].set_title("Predicted vs Actual", fontsize=12)
axs[0, 0].set_xlabel("Predicted Salary")
axs[0, 0].set_ylabel("Actual Salary")

sns.scatterplot(x="prediction", y="residual", data=pred_pd, color="skyblue", alpha=0.6, ax=axs[0, 1])
sns.regplot(x="prediction", y="residual", data=pred_pd, scatter=False, ax=axs[0, 1], color="darkred", line_kws={"lw":2})
axs[0, 1].axhline(0, color="gray", linestyle="--")
axs[0, 1].set_title("Residuals vs Predicted", fontsize=12)
axs[0, 1].set_xlabel("Predicted Salary")
axs[0, 1].set_ylabel("Residuals")

sns.histplot(pred_pd["residual"], kde=True, bins=30, color="cornflowerblue", ax=axs[1, 0], edgecolor="black")
sns.kdeplot(pred_pd["residual"], color="darkred", lw=2, ax=axs[1, 0])
axs[1, 0].set_title("Histogram of Residuals")
axs[1, 0].set_xlabel("Residuals")

stats.probplot(pred_pd["residual"], dist="norm", plot=axs[1, 1])
axs[1, 1].get_lines()[1].set_color("darkred")
axs[1, 1].get_lines()[1].set_linewidth(2)
axs[1, 1].set_title("QQ Plot of Residuals")

plt.tight_layout()
plt.savefig("_output/diagnostic_plots.png", dpi=300)
plt.close()

![Diagnostic Plots](./_output/diagnostic_plots.png)

Predicted vs Actual Plot (Top Left):
The data points mostly align with the red reference line (y = x), indicating that the model predicts salaries well. The linear trend suggests a strong fit between predicted and actual values.

Residuals vs Predicted Plot (Top Right):
Residuals are randomly scattered around the zero line, supporting the assumption of linearity and indicating that the model does not suffer from major systematic bias. However, there is some heteroscedasticity (increasing spread at higher predictions) which might suggest variance is not constant.

Histogram of Residuals (Bottom Left):
The distribution of residuals is approximately normal, though slightly left-skewed. This supports the assumption of normally distributed errors, which is key for statistical inference validity.

QQ Plot of Residuals (Bottom Right):
Most points fall along the red reference line, confirming that the residuals follow a normal distribution. Deviations in the tails may indicate mild outliers or slight non-normality at the extremes.

# Evaluation

The evaluation of the model is crucial to understand its performance. In this section, we will calculate and visualize the following metrics:
1. **R² (Coefficient of Determination)**: Indicates how well the model explains the variance in the target variable.
2. **RMSE (Root Mean Squared Error)**: Measures the average magnitude of the errors between predicted and actual values.

In [30]:
#| eval: true
#| echo: false
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, pow, sqrt, avg
import numpy as np

test_results = lr_model.evaluate(test_data)

r2 = test_results.r2
rmse = test_results.rootMeanSquaredError

print("R² (R-squared):", r2)
print("RMSE (Root Mean Squared Error):", rmse)

import matplotlib.pyplot as plt

metrics = ["R²", "RMSE"]
values = [r2, rmse]

plt.figure(figsize=(6, 4))
bars = plt.bar(metrics, values, color=["seagreen", "steelblue"])
plt.title("Model Evaluation Metrics", fontsize=14)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.01, f"{yval:.3f}", ha='center', va='bottom')

plt.ylabel("Value")
plt.ylim(0, max(values) * 1.1)
plt.savefig("_output/model_metrics_barplot.png", dpi=300)
plt.close()

R² (R-squared): 0.8412012932179281
RMSE (Root Mean Squared Error): 17326.42176937338


![Model Evaluation Metrics](./_output/model_metrics_barplot.png)

The model achieved an R² score of 0.841, which means that approximately 84.1% of the variance in the target variable (salary) is explained by the selected features in the linear regression model. This indicates a strong fit and high explanatory power.

The RMSE is 17,326, which represents the average magnitude of the prediction errors. This means, on average, the model's predicted salary differs from the actual salary by around $17,326.

## Model Evaluation Plot

- Display the predicted vs actual salary plot with a red line indicating the ideal fit (y=x).
- Use `seaborn` or `matplotlib` to create the plot.
- Customize the plot with appropriate titles, labels, and legends.
- Describe the plot in a few sentences, highlighting key insights and observations.

In [27]:
#| eval: true
#| echo: false
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(7, 6))
sns.scatterplot(x="prediction", y="label", data=pred_pd, alpha=0.5, color="skyblue", label="Predictions")

max_val = max(pred_pd["prediction"].max(), pred_pd["label"].max())
min_val = min(pred_pd["prediction"].min(), pred_pd["label"].min())
plt.plot([min_val, max_val], [min_val, max_val], color="darkred", linestyle="--", linewidth=2, label="Ideal Fit (y = x)")

plt.title("Predicted vs Actual Salary", fontsize=14)
plt.xlabel("Predicted Salary")
plt.ylabel("Actual Salary")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("_output/predicted_vs_actual.png", dpi=300)
plt.close()

![Diagnostic Plots](./_output/predicted_vs_actual.png)

This plot compares the predicted salaries (x-axis) to the actual salaries (y-axis) in the test dataset.
The dashed red line represents the ideal scenario where predictions perfectly match actual values.

Most data points are clustered closely around the red line, which indicates that the model has a strong ability to estimate salary.
Some points deviate from the line, especially in the higher salary range, suggesting that predictions tend to slightly under- or overestimate in extreme cases.

# Submission {.unnumbered}
1. Save figures in the `_output/` folder.
2. Commit and push code and output files:
```bash
git add .
git commit -m "Add Lab 08 Salary Prediction models and output"
git push origin main
```
3. Submit your GitHub repository link.

# Resources {.unnumbered}
- [PySpark MLlib Docs](https://spark.apache.org/docs/latest/ml-guide.html)  
- [Seaborn Docs](https://seaborn.pydata.org/)  
- [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html)
